<a href="https://colab.research.google.com/github/motazsaad/infant-cry-care/blob/main/FastApiCryCare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
import librosa 
import matplotlib.pyplot as plt
import numpy as np
from pydantic import BaseModel
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from fastapi.encoders import jsonable_encoder
from fastapi.responses import JSONResponse
classes=['Hungry','Sleepy','Uncomfortable','asphyxia','burp up','lower gas','normal','pain']
mainPath = '/content/drive/MyDrive/CryCareDataset/'
exp4Path = mainPath + 'generatedData/Exp4/'
filename = mainPath + 'cnn.sav'
cnn = pickle.load(open(filename, 'rb'))


In [ ]:
app = FastAPI()
class Item(BaseModel):
    path: str

@app.post('/predictAudio')
async def predictAudio(item: Item):
  
  audio = item.path
  audiofile, sr = librosa.load(audio, sr = 11025, mono = True)
  plt.specgram(audiofile, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap='CMRmap', sides='default', mode='default', scale='dB');
  plt.axis('off');
  name = audio.split('/')[-1]
  filaname = f'{mainPath}predictedImage/{name[:-3].replace(".", "")}.png'
  plt.savefig(filaname)

  my_image = load_img(filaname, target_size=(256, 256))
  my_image = img_to_array(my_image)
  my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
  # my_image = preprocess_input(my_image)

  y = cnn.predict(my_image)
  predicted_class = np.argmax(np.round(y),axis=1)[0]
  return {
      "label" : classes[predicted_class]
      }

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)